**Parent kernal [link ](https://www.kaggle.com/julian3833/g-research-starter-lgbm-pipeline-lb)**

**changes n_estimators=1000,num_leaves=500,max_depth=10**

In [6]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import gresearch_crypto


TRAIN_CSV = '/kaggle/input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '/kaggle/input/g-research-crypto-forecasting/asset_details.csv'

In [7]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [8]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV).sort_values("Asset_ID")
df_asset_details

,Asset_ID,Weight,Asset_Name
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
0,2,2.397895,Bitcoin Cash
10,3,4.406719,Cardano
13,4,3.555348,Dogecoin
3,5,1.386294,EOS.IO
5,6,5.894403,Ethereum
4,7,2.079442,Ethereum Classic
11,8,1.098612,IOTA
6,9,2.397895,Litecoin


# Training

## Utility functions to train a model for one asset

## Loop over all assets

In [9]:
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
# Two new features from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

# A utility function to build features from the original df
# It works for rows to, so we can reutilize it.
def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['upper_Shadow'] = upper_shadow(df_feat)
    df_feat['lower_Shadow'] = lower_shadow(df_feat)
    df_feat["high_div_low"] = df_feat["High"] / df_feat["Low"]
    #df_feat["open_sub_close"] = df_feat["Open"] - df_feat["Close"]
    df_feat['trade']=df_feat['Close']-df_feat['Open']
    df_feat['gtrade']=df_feat['trade']/df_feat['Count']
    df_feat['shadow1']=df_feat['trade']/df_feat['Volume']
    #df_feat['shadow2']=df_feat['upper_Shadow']/df['Low']
    df_feat['shadow3']=df_feat['upper_Shadow']/df_feat['Volume']
    #df_feat['shadow4']=df_feat['lower_Shadow']/df['High']
    df_feat['shadow5']=df_feat['lower_Shadow']/df_feat['Volume']
    
    df_feat['diff1']=df_feat['Volume']-df_feat['Count']
    
    df_feat['mean1']=(df_feat['shadow5']+df_feat['shadow3'])/2
    
    df_feat['mean2']=(df_feat['shadow1']+df_feat['Volume'])/2
    
    df_feat['mean3']=(df_feat['trade']+df_feat['gtrade'])/2
    
    df_feat['mean4']=(df_feat['diff1']+df_feat['upper_Shadow'])/2
    
    df_feat['mean5']=(df_feat['diff1']+df_feat['lower_Shadow'])/2 
    
    return df_feat
def add_feat_nn(X,asset):
    pth=f'../input/autoencoderfeatures/{asset}.npy'
    extra=np.load(pth)

    
    conc=np.concatenate([X,extra],axis=-1)
    return conc
def log(model,X_train, X_valid, y_train, y_valid,train_split=1.0):
    if train_split > 0:
        X_train=X_train[:int(train_split*X_train.shape[0])]
        y_train=y_train[:int(train_split*y_train.shape[0])]
    
        pred=model.predict(X_train)
        print('Training :- ')
        print(f'MSE : {np.mean((y_train-pred)**2)}')
        print(f'CV : {pearsonr(pred,y_train)[0]}')
    pred=model.predict(X_valid)
    print('Validation :- ')
    print(f'MSE : {np.mean((y_valid-pred)**2)}')
    print(f'CV : {pearsonr(pred,y_valid)[0]}')

def get_Xy_and_model_for_asset(df_train, asset_id):
    df = df_train[df_train["Asset_ID"] == asset_id]
   
    # TODO: Try different features here!
    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop("y", axis=1)
    y = df_proc["y"]
    X=np.array(X)
    print(y.shape)
    
    X=add_feat_nn(X,asset_id)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    # TODO: Try different models here!
    model = LGBMRegressor(n_estimators=200,num_leaves=50,learning_rate=0.09)
    model.fit(X_train, y_train)
    print('[Finished Training] evaluating')
    log(model,X_train, X_test, y_train, y_test,0.3)
    
    
    return model

In [10]:
Xs = {}
ys = {}
models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    model = get_Xy_and_model_for_asset(df_train, asset_id)    
    models[asset_id] = model

Training model for Binance Coin     (ID=0 )
(1929204,)
[Finished Training] evaluating
Training :- 
MSE : 2.650809410286785e-05
CV : 0.37962671284077987
Validation :- 
MSE : 3.0006025637692826e-05
CV : 0.08159123297113066
Training model for Bitcoin          (ID=1 )
(1955978,)
[Finished Training] evaluating
Training :- 
MSE : 3.7645244316050262e-06
CV : 0.3088802658695261
Validation :- 
MSE : 3.974023466532083e-06
CV : 0.1384173111649221
Training model for Bitcoin Cash     (ID=2 )
(1948676,)
[Finished Training] evaluating
Training :- 
MSE : 3.09481743519305e-05
CV : 0.574716239669476
Validation :- 
MSE : 4.367000456874614e-05
CV : 0.026298845134861335
Training model for Cardano          (ID=3 )
(1773136,)
[Finished Training] evaluating
Training :- 
MSE : 1.933610630963507e-05
CV : 0.27271130581569436
Validation :- 
MSE : 2.0143933384481396e-05
CV : 0.09226993524513763
Training model for Dogecoin         (ID=4 )
(1011892,)
[Finished Training] evaluating
Training :- 
MSE : 5.42914310644352

In [11]:
# Check the model interface
x = get_features(df_train.iloc[1])
y_pred = models[0].predict([x])
y_pred[0]

ValueError: Number of features of the model must match the input. Model n_features_ is 85 and input n_features is 21 

# Predict & submit

References: [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

Something that helped me understand this iterator was adding a pdb checkpoint inside of the for loop:

```python
import pdb; pdb.set_trace()
```

See [Python Debugging With Pdb](https://realpython.com/python-debugging-pdb/) if you want to use it and you don't know how to.


In [ ]:
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

for i, (df_test, df_pred) in enumerate(iter_test):
    for j , row in df_test.iterrows():
        
        model = models[row['Asset_ID']]
        x_test = get_features(row)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
        
        
        # Print just one sample row to get a feeling of what it looks like
        if i == 0 and j == 0:
            display(x_test)

    # Display the first prediction dataframe
    if i == 0:
        display(df_pred)

    # Send submissions
    env.predict(df_pred)